# Primeros pasos con Spark

## SparkContext vs SparkSession

SparkContext es el punto de entrada a Spark desde las versiones 1.x y se utiliza para crear de forma programativa RDD, acumuladores y variables broadcast en el clúster. Desde Spark 2.0, la mayoría de funcionalidades (métodos) disponibles en SparkContext también los están en SparkSession. Su objeto sc está disponible en el spark-shell y se puede crear de forma programativa mediante la clase SparkContext.

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

SparkSession se introdujo en la versión 2.0 y es el punto de entrada para crear RDD, DataFrames y DataSets. El objeto spark se encuentra disponible por defecto en el spark-shell y se puede crear de forma programativa mediante el patrón builder de SparkSession.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

Además, desde una sesión de Spark podemos obtener un contexto a través de la propiedad sparkContext:

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Hola Spark

Lo primero que debemos hacer siempre es conectarnos a la sesión de Spark, el cual le indica a Spark como acceder al clúster. Si utilizamos la imagen de Docker, debemos obtener siempre la sesión a partir de la clase SparkSession:

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() # SparkSession de forma programativa
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
sc = spark.sparkContext                    # SparkContext a partir de la sesión

In [0]:
# Suma de los 100 primeros números
rdd = sc.parallelize(range(100 + 1))
rdd.sum()

Out[6]: 5050

En cambio, si utilizamos la instalación de PySpark (o la solución de Databricks) que tenemos en la máquina virtual, directamente podemos acceder a la instancia de SparkSession a través del objeto global spark:

In [0]:
sc = spark.sparkContext     # spark es una instancia de la clase SparkSession

In [0]:
rdd = sc.parallelize(range(100 + 1))
rdd.sum()

Out[8]: 5050

## Arquitectura

Ya hemos comentado que Spark es un sistema distribuido diseñado para procesar grandes volúmenes de datos de forma rápida y eficiente. Este sistema normalmente se despliega en un conjunto de máquinas que se conocen como un clúster Spark, pudiendo estar compuesta de unas pocas máquinas o miles de ellas. Según el FAQ de Spark, el clúster más grande de Spark contiene más de 8000 nodos.

A la hora del despliegue, se utiliza un sistema de gestión de recursos como el gestor propio de Spark (conocido como Spark Standalone), Apache Mesos, Kubernetes o YARN para gestionar de forma inteligente y eficiente el clúster.

Los dos componentes principales del clúster son:

- el gestor del clúster: nodo maestro que sabe donde se localizan los trabajadores, cuanta memoria disponen y el número de cores CPU de cada nodo. Su mayor responsabilidad es orquestar el trabajo asignándolo a los diferentes nodos.

- los nodos trabajadores (workers): cada nodo ofrece recursos (memoria, CPU, etc...) al gestor del clúster y realiza las tareas que se le asignen.

### Aplicaciones Spark

Una aplicación Spark se compone de dos partes:

1. La lógica de procesamiento de los datos, la cual realizamos mediante alguna de las API que ofrece Spark (Java, Scala, Python, etc...), desde algo sencillo que realice una ETL sobre los datos a problemas más complejos que requieran múltiples iteraciones y tarden varias horas como entrenar un modelo de machine learning.

2. Driver: es el coordinador central encargado de interactuar con el clúster Spark y averiguar qué máquinas deben ejecutar la lógica de procesamiento. Para cada una de esas máquinas, el driver realiza una petición al clúster para lanzar un proceso conocido como ejecutor (executor). Además, el driver Spark es responsable de gestionar y distribuir las tareas a cada ejecutor, y si es necesario, recoger y fusionar los datos resultantes para presentarlos al usuario. Estas tareas se realizan a través de la SparkSession.

Cada ejecutor es un proceso JVM (Java Virtual Machine) dedicado para una aplicación Spark específica. Un ejecutor vivirá tanto como dure la aplicación Spark, lo cual puede ser segundos, minutos o días, dependiendo de la complejidad de la aplicación. Conviene destacar que los ejecutores son elementos aislados que no se comparten entre aplicaciones Spark, por lo que la única manera de compartir información entre diferente ejecutores es mediante un sistema de almacenamiento externo como HDFS o S3.

![](https://community.cloud.databricks.com/files/shared_uploads/notebook_1/1.jpg)

Así pues, Spark utiliza una arquitectura maestro/esclavo, donde el driver es el maestro, y los ejecutores los esclavos. Cada uno de estos componentes se ejecutan como un proceso independiente en el clúster Spark. Por lo tanto, una aplicación Spark se compone de un driver y múltiples ejecutores. Cada ejecutor realiza lo que se le pide en forma de tareas, ejecutando cada una de ellas en un núcleo CPU separado. Así es como el procesamiento paralelo acelera el tratamiento de los datos. Además, cada ejecutor, bajo petición de la lógica de la aplicación, se responsabiliza de cachear un fragmento de los datos en memoria y/o disco.

Al lanzar una aplicación Spark, podemos indicar el número de ejecutores que necesita la aplicación, así como la cantidad de memoria y número de núcleos que debería tener cada ejecutor.

![2](https://community.cloud.databricks.com/files/Notebook/2.png)


### Job, Stage y Task

Cuando creamos una aplicación Spark, por debajo, se distinguen los siguientes elementos:

- Job (trabajo): computación paralela compuesta de múltiples tareas que se crean tras una acción de Spark (save, collect, etc...). Al codificar nuestro código mediante PySpark, el driver convierte la aplicación Spark en uno o más jobs, y a continuación, estos jobs los transforma en un DAG (grafo). Este grafo, en esencia, es el plan de ejecución, donde cada elemento dentro del DAG puede implicar una o varias stages (escenas).

- Stage (escena): cada job se divide en pequeños conjuntos de tareas que forman un escenario. Como parte del grafo, las stages se crean a partir de si las operaciones se pueden realizar de forma paralela o de forma secuencial. Como no todas las operaciones pueden realizarse en una única stage, en ocasiones se dividen en varias, normalmente debido a los límites computacionales de los diferentes ejecutores.

- Task (tarea): unidad de trabajo más pequeña que se envía a los ejecutores Spark. Cada escenario se compone de varias tareas. Cada una de las tareas se asigna a un único núcleo y trabaja con una única partición de los datos. Por ello, un ejecutor con 16 núcleos puede tener 16 o más tareas trabajando en 16 o más particiones en paralelo.

![3](https://community.cloud.databricks.com/files/Notebook/3.png)


### DataFrame

La principal abstracción de los datos en Spark es el Dataset. Se pueden crear desde las fuentes de entrada de Hadoop (como ficheros que provienen de HDFS o S3) o mediante transformaciones de otros Datasets. Dado el cariz de Python, no necesitamos que los Dataset estén fuertemente tipados, por eso, todos los Dataset que usemos serán Dataset[Row] (si trabajásemos mediante Java o Scala sí deberíamos indicar el tipo de sus datos), y por consistencia con el concepto de Pandas y R, los llamaremos DataFrame.

Por ejemplo, veamos cómo podemos crear un DataFrame a partir de un fichero de texto:

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() 

quijoteTxt = spark.read.text("dbfs:/FileStore/Notebook/el_quijote.txt")
quijoteTxt.count()  # número de filas del DataFrame - 2186


Out[9]: 2186

#### Explicar linea a linea el codigo de arriba

In [0]:
quijoteTxt.first()  # primera fila - Row(value='DON QUIJOTE DE LA MANCHA')

Out[10]: Row(value='DON QUIJOTE DE LA MANCHA')

In [0]:
# Transformamos un DataFrame en otro nuevo
lineasConQuijote = quijoteTxt.filter(quijoteTxt.value.contains("Quijote"))  # DataFrame con las líneas que contiene la palabra Quijote
lineasConQuijote.count()  # cantidad de líneas con la palabra Quijote - 584

Out[11]: 584

In [0]:
# Las transformaciones se pueden encadenar
quijoteTxt.filter(quijoteTxt.value.contains("Quijote")).count()     # idem - 584

Out[12]: 584

Estudiaremos los DataFrame en profundidad en las próximas clases

### Spark UI (Jupyter Lab)

Si accedemos a la dirección 'http://127.0.0.1:4040/', veremos un interfaz gráfico donde podemos monitorizar y analizar el código Spark ejecutado. La barra superior muestra un menú con las opciones para visualizar los jobs, stages, el almacenamiento, el entorno y sus variables de configuración, y finalmente los ejecutores:

![4](https://community.cloud.databricks.com/files/Notebook/4.png)

![5](https://community.cloud.databricks.com/files/Notebook/5.png)

### Spark UI en Databricks

Para acceder a la herramienta de monitorización en Databricks, una vez creado un clúster, en la opción Compute/Calcular podremos seleccionar el clúster creado y en la pestaña IU de Spark acceder al mismo interfaz gráfico:

![](https://community.cloud.databricks.com/files/shared_uploads/notebook_1/6.png)

# Actividad

En otro notebook hacer los ejemplos de código anteriores utilizando Scala

In [0]:
%scala
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()

val quijoteTxt = spark.read.text("dbfs:/FileStore/Notebook/el_quijote.txt")
val rowCount = quijoteTxt.count()  // número de filas del DataFrame - 2186

println(s"Número de filas: $rowCount")


Número de filas: 2186
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a115020
quijoteTxt: org.apache.spark.sql.DataFrame = [value: string]
rowCount: Long = 2186

In [0]:
%scala
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()

val quijoteTxt = spark.read.text("dbfs:/FileStore/Notebook/el_quijote.txt")
val firstRow = quijoteTxt.first()  // primera fila - Row(value='DON QUIJOTE DE LA MANCHA')

println(s"Primera fila: $firstRow")

Primera fila: [DON QUIJOTE DE LA MANCHA]
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a115020
quijoteTxt: org.apache.spark.sql.DataFrame = [value: string]
firstRow: org.apache.spark.sql.Row = [DON QUIJOTE DE LA MANCHA]

In [0]:
%scala
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()

val quijoteTxt = spark.read.text("dbfs:/FileStore/Notebook/el_quijote.txt")
val lineasConQuijote = quijoteTxt.filter(quijoteTxt("value").contains("Quijote"))  // DataFrame con las líneas que contiene la palabra "Quijote"
val countLineasConQuijote = lineasConQuijote.count()  // cantidad de líneas con la palabra Quijote - 584

println(s"Cantidad de líneas con la palabra 'Quijote': $countLineasConQuijote")


Cantidad de líneas con la palabra 'Quijote': 584
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a115020
quijoteTxt: org.apache.spark.sql.DataFrame = [value: string]
lineasConQuijote: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [value: string]
countLineasConQuijote: Long = 584

In [0]:
%scala
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()

val quijoteTxt = spark.read.text("dbfs:/FileStore/Notebook/el_quijote.txt")
val countLineasConQuijote = quijoteTxt.filter(quijoteTxt("value").contains("Quijote")).count()  // cantidad de líneas con la palabra Quijote - 584

println(s"Cantidad de líneas con la palabra 'Quijote': $countLineasConQuijote")


Cantidad de líneas con la palabra 'Quijote': 584
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a115020
quijoteTxt: org.apache.spark.sql.DataFrame = [value: string]
countLineasConQuijote: Long = 584

# RDD

Un RDD (Resilient Distributed Datasets) es una estructura de datos que abstrae los datos para su procesamiento en paralelo.

Antes de Spark 2.0, los RDD eran el interfaz principal para interactuar con los datos.

Se trata de una colección de elementos tolerantes a fallos que son immutables (una vez creados, no se pueden modificar) y diseñados para su procesamiento distribuido. Cada conjunto de datos en los RDD se divide en particiones lógicas, que se pueden calcular en diferentes nodos del clúster.

Hay dos formas de crear un RDD:

- Paralelizando una colección ya existente en nuestra aplicación Spark.

- Referenciando un dataset de un sistema externo como HDFS, HBase, etc...

Sobre los RDD se pueden realizar dos tipos de operaciones:

- Acción: devuelven un valor tras ejecutar una computación sobre el conjunto de datos.

- Transformación: es una operación perezosa (lazy) que crea un nuevo conjunto de datos a partir de otro RDD/Dataset, tras realizar un filtrado, join, etc...

> **¿ Esta obsoleto RDD?**
> 
> Antes de la versión 2.0, el principal interfaz para programar en Spark eran los RDD. Tras la versión 2.0, fueron sustituidos por los Dataset, que son RDD fuertemente tipados que además están optimizados a bajo nivel. Aunque el interfaz RDD todavía tiene soporte, sin embargo, se recomienda el uso de los Dataset por su mejor rendimiento.
> Contexto y evolución
RDD (Resilient Distributed Dataset): Es la abstracción fundamental que Apache Spark introdujo inicialmente. Los RDDs son colecciones distribuidas de objetos que pueden ser procesadas en paralelo. Proporcionan una API de bajo nivel y permiten un control detallado sobre las transformaciones y acciones en el procesamiento de datos distribuidos.

> DataFrame API: Introducida en Spark 1.3, la API de DataFrame se basa en el concepto de tablas de bases de datos. Ofrece una capa de abstracción más alta que los RDDs y permite realizar operaciones de manera declarativa. Los DataFrames también ofrecen optimizaciones automáticas gracias al Catalyst Optimizer y el uso de Tungsten para la gestión de memoria, lo que hace que el procesamiento sea más eficiente.

>Dataset API: Introducida en Spark 1.6 y diseñada para el uso en Scala y Java, combina las ventajas de los RDDs (tipos de datos específicos y seguridad de tipo) con las optimizaciones de los DataFrames. La API de Dataset es más fuerte en cuanto a tipo, lo que mejora la seguridad de tipo y reduce errores en tiempo de compilación.
>¿Por qué se prefieren DataFrames y Datasets sobre RDDs?
>Optimización: Las APIs de DataFrame y Dataset son optimizadas automáticamente por el Catalyst Optimizer, lo que permite ejecutar código de forma mucho más eficiente.

> Facilidad de uso: DataFrames y Datasets proporcionan una API de más alto nivel, que es más intuitiva y se asemeja al lenguaje SQL, haciéndolas más accesibles para usuarios no especializados en programación funcional.

> Mejor manejo de memoria: La gestión de memoria en DataFrames y Datasets está mejorada por el proyecto Tungsten, que reduce el consumo de memoria y mejora el rendimiento general de Spark.

> Compatibilidad con SQL: Los DataFrames y Datasets están diseñados para trabajar de manera fluida con SQL, lo cual facilita el análisis y manipulación de datos estructurados.

> ¿Cuándo usar RDDs?
> Aunque los DataFrames y Datasets son generalmente recomendados, los RDDs aún son útiles en ciertas situaciones:

> Procesamiento de datos no estructurados o semiestructurados: Cuando necesitas realizar operaciones complejas en datos que no pueden ser representados fácilmente como DataFrames o Datasets, los RDDs pueden ser una opción.
Transformaciones personalizadas: Si necesitas un control detallado sobre las transformaciones o si las optimizaciones automáticas de Catalyst no son aplicables, RDDs te ofrecen más control.
API de bajo nivel: Los RDDs proporcionan una API de bajo nivel que puede ser útil para desarrolladores avanzados que necesitan manipular datos distribuidos de forma muy específica.

## Acciones

En Apache Spark, las acciones son operaciones que se ejecutan sobre RDDs, DataFrames o Datasets y que desencadenan el cómputo real en el clúster. Es decir, las acciones inician el procesamiento y devuelven un valor o resultado al controlador del programa o escriben datos a un almacenamiento externo.

Las acciones contrastan con las transformaciones, que son operaciones que definen cómo deben transformarse los datos pero no ejecutan inmediatamente el procesamiento. Las transformaciones son "perezosas" (lazy), mientras que las acciones desencadenan la ejecución.

A continuación vamos a revisar las acciones más comunes. Puedes consultar todas las acciones disponibles en la  [documentación oficial](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions)
de Spark:

> Recuerda importar el contexto:

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Parallelize

En Apache Spark, parallelize es un método utilizado para crear un RDD (Resilient Distributed Dataset) a partir de una colección de datos en el controlador del programa. Permite distribuir los datos en paralelo entre los nodos del clúster para realizar operaciones distribuidas.



Podemos crear RDD directamente desde cero sin necesidad de leer los datos desde un fichero. Para ello, a partir de un `SparkContext` podemos utilizar `parallelize` sobre una lista.

Esta acción divide una colección de elementos entre los nodos de nuestro clústers. Por ejemplo:

In [0]:
miRDD = sc.parallelize([1,2,3,4,5,6,7,8,9])

lista  = ['Hola', 'Adiós', 'Hasta luego']
listaRDD = sc.parallelize(lista) # Creamos un RDD a partir de una lista
listaRDD4 = sc.parallelize(lista,4) # Creamos un RDD con 4 particiones


Crear un RDD de una lista de números:

``
miRDD = sc.parallelize([1,2,3,4,5,6,7,8,9])
``

Conviertes la lista [1, 2, 3, 4, 5, 6, 7, 8, 9] en un RDD (conjunto de datos distribuido) para que Spark pueda trabajar con esos datos de forma paralela.
Crear un RDD de una lista de palabras:

``
lista = ['Hola', 'Adiós', 'Hasta luego']
listaRDD = sc.parallelize(lista)
``

Haces lo mismo, pero esta vez con una lista de palabras. listaRDD es un RDD que contiene ['Hola', 'Adiós', 'Hasta luego'].
Crear un RDD con varias particiones:

``
listaRDD4 = sc.parallelize(lista, 4)
`` 

Este RDD se crea a partir de la misma lista, pero se divide en 4 partes para que se pueda procesar en paralelo de manera más eficiente.

In [0]:
lista

Out[19]: ['Hola', 'Adiós', 'Hasta luego']

In [0]:
listaRDD

Out[21]: ParallelCollectionRDD[45] at readRDDFromInputStream at PythonRDD.scala:435

>La primera salida (Out[19]) muestra los datos reales que se han extraído del RDD utilizando un método como .collect(), y se presentan como una lista de Python.

>La segunda salida (Out[21]) indica que el resultado es un RDD que Spark está manejando de forma distribuida. No contiene los datos en sí, sino una referencia al RDD.

In [0]:
listaRDD4

Out[22]: ParallelCollectionRDD[46] at readRDDFromInputStream at PythonRDD.scala:435

> La salida Out[13]: ParallelCollectionRDD[29] at readRDDFromInputStream at PythonRDD.scala:435 indica que Spark ha creado un RDD (un conjunto de datos distribuido).

>ParallelCollectionRDD[29]: Es un RDD que se creó usando parallelize(), y [29] es su identificador.
at readRDDFromInputStream at PythonRDD.scala:435: Muestra en qué parte del código de Spark se generó el RDD, algo que ayuda a los desarrolladores de Spark para depurar.

### Take y Sample

Son métodos utilizados en RDDs, DataFrames y Datasets para interactuar con los datos. Ambos sirven para trabajar con subconjuntos, pero tienen usos y características diferentes. Una vez tenemos un RDD, y sobre él queremos recuperar un número determinado de resultados, de forma similar a limit en SQL, tenemos la acción `take`:

In [0]:
miRDD.take(3)       # [1, 2, 3]]

Out[23]: [1, 2, 3]

### Explicación del Código miRDD.take(3)

``
miRDD.take(3)  # [1, 2, 3]
``

miRDD: Es un RDD creado previamente, posiblemente a partir de una lista de números (por ejemplo, [1, 2, 3, 4, 5, 6, 7, 8, 9]).
take(3): Este método devuelve una lista con los primeros 3 elementos del RDD miRDD.

In [0]:
listaRDD.take(2)    # ['Hola', 'Adiós']

Out[25]: ['Hola', 'Adiós']

Otra opción es utilizar `sample` para obtener una muestra de los datos, aunque en este caso no es una acción sino una transformación, ya que va a obtener información posiblemente de varias particiones, teniendo que barajar los datos (**shuffle**):

In [0]:
miRDDmuestra = miRDD.sample(False, 0.5)

# Explicación del Código miRDDmuestra 

``
miRDDmuestra = miRDD.sample(False, 0.5)
``

Descripción
miRDD:

Es un RDD (Resilient Distributed Dataset), que es una estructura de datos utilizada en Spark para distribuir y paralelizar el procesamiento de datos.
sample(False, 0.5):

El método .sample(withReplacement, fraction) se utiliza para obtener una muestra aleatoria del RDD.
withReplacement=False:
Indica que se tomará la muestra sin reemplazo, lo cual significa que cada elemento puede seleccionarse solo una vez.
fraction=0.5:
Indica que se desea seleccionar aproximadamente el 50% de los elementos del RDD miRDD.
miRDDmuestra:

Es un nuevo RDD que contiene la muestra aleatoria de los elementos de miRDD.
Al ser una muestra aleatoria, cada ejecución podría dar resultados diferentes, ya que la selección de los elementos se realiza de forma aleatoria.

In [0]:
miRDDmuestra.collect()  # [2, 4, 6, 7, 8, 9] / [1, 2, 3, 4, 6] / [5, 8, 9]

Out[27]: [2, 4]

# Explicación del Código miRDDmuestra.collect()


``
miRDDmuestra.collect()  # [2, 4, 6, 7, 8, 9] / [1, 2, 3, 4, 6] / [5, 8, 9]
``

Descripción
miRDDmuestra: Es un RDD que contiene una muestra aleatoria de los datos originales.

.collect(): Recolecta todos los elementos del RDD y los devuelve como una lista de Python.

Esta transformación recibe varios parámetros:

- `withReplacement`: booleano para indicar si queremos elementos repetidos
- `fraction` : valor entre 0 y 1 que expresa la probabilidad de elegir cada elemento
- opcionalmente se le puede pasar un tercer valor con la semilla

Así pues, en el ejemplo anterior, cada llamada a sample ha generado un RDD diferente, sin valores repetidos, pero con un tamaño de RDD variable.

**Muestra Estratificada**  

Si necesitamos que nuestra muestra esté estratificada para que los datos no estén sesgados podemos utilizar `sampleByKey`:

¿Qué es una muestra estratificada en este contexto?

In [0]:
datos = sc.parallelize([(1, 'a'), (1, 'b'), (2, 'c'), (2, 'd'), (2, 'e'), (3, 'f')])

# indicamos la probabilidad para cada clave
fracciones = {1: 0.1, 2: 0.6, 3: 0.3}

approxSample = datos.sampleByKey(False, fracciones)     # [(2, 'c'), (2, 'd')] el resultado puede variar
print(approxSample.collect())

[(2, 'c'), (2, 'd'), (2, 'e')]


Para obtener una muestra mediante una acción, tenemos la opción `takeSample` que funciona de forma similar pero sin hacer `shuffle` y devuelve una lista:

In [0]:
miRDDmuestraT = miRDD.takeSample(False, 5)
print(miRDDmuestraT)  # [1, 8, 9, 7, 2] el resultado puede variar

[9, 4, 8, 7, 3]


El primer parámetro vuelve a indicar si hay repetidos, pero el segundo fija la cantidad de elementos a devolver.

Por último, mediante `top` obtenemos los primeros elementos una vez ordenado el RDD:

In [0]:
miRDD.top(3)    # [9, 8, 7]

Out[30]: [9, 8, 7]

De forma similar, tenemos `takeOrdered` que recupera la cantidad de registros necesarios pero ordenados ascendentemente (al contrario que `top`), con la opción de ordenarlos descendentemente (similar a `top`):

In [0]:
miRDD.takeOrdered(3)    # [1, 2, 3]

Out[31]: [1, 2, 3]

In [0]:
miRDD.takeOrdered(3, lambda x: -x)    # [9, 8, 7]

Out[32]: [9, 8, 7]

# Explicación de las Funciones Lambda en Python

## ¿Qué es una Función Lambda?

Las **funciones lambda** en Python son **funciones anónimas** y pequeñas que se definen utilizando la palabra clave `lambda`. Se usan cuando necesitas una función corta y simple para realizar una operación específica, y no quieres definir una función completa con `def`.

### Sintaxis

```python
lambda argumentos: expresión


#Ejemplo 1: Sumar dos números

In [0]:
suma = lambda x, y: x + y
print(suma(3, 5))  # Salida: 8

8


#Ejemplo 2: Calcular el doble de un número

In [0]:
numeros = [1, 2, 3, 4]
dobles = list(map(lambda x: x * 2, numeros))
print(dobles)  # Salida: [2, 4, 6, 8]


[2, 4, 6, 8]


#Ejemplo 3: Usar lambda con map()

In [0]:
numeros = [1, 2, 3, 4]
dobles = list(map(lambda x: x * 2, numeros))
print(dobles)  # Salida: [2, 4, 6, 8]


[2, 4, 6, 8]


Hay que tener cuidado si el conjunto de datos es muy grande, porque tanto `take` como `takeSample`, `takeOrdered` y `top` llevarán todos los datos a memoria.

> ¿Que alternativas tengo?

### Alternativas a `take()`, `takeSample()`, `takeOrdered()`, y `top()` en Spark

Cuando trabajas con grandes volúmenes de datos en Spark, funciones como `take()`, `takeSample()`, `takeOrdered()`, y `top()` pueden ser problemáticas, ya que intentan llevar todos los datos al **nodo controlador**, lo cual puede agotar la memoria. Aquí te presentamos algunas alternativas eficientes para manejar conjuntos de datos grandes sin comprometer el rendimiento.

##### 1. sample()
Utiliza **`sample(withReplacement, fraction)`** para trabajar con una **muestra representativa** de los datos sin procesar el conjunto completo.


In [0]:
muestraRDD = miRDD.sample(False, 0.1)  # Toma el 10% de los datos

####2. take(n) Combinado con filter()
Reduce el tamaño del conjunto de datos antes de aplicar take() utilizando filter() para obtener solo una parte específica de los datos.

In [0]:
filtroRDD = miRDD.filter(lambda x: x % 2 == 0)  # Filtra los elementos pares
primeros = filtroRDD.take(10)  # Toma solo 10 elementos después de filtrar


####3. count() y countApprox()
Para contar elementos, usa count() o countApprox() si necesitas una aproximación más rápida.

In [0]:
total = miRDD.count()  # Cuenta todos los elementos
totalAprox = miRDD.countApprox(5000)  # Cuenta de manera aproximada en 5000 ms


####4. reduce()
Para resumir los datos sin traerlos todos al controlador, utiliza reduce(), lo cual permite aplicar una operación de reducción (como una suma).

In [0]:
suma = miRDD.reduce(lambda a, b: a + b)  # Suma todos los elementos del RDD


####5. foreach()
Si deseas aplicar una operación a cada elemento del RDD sin traer todos los datos al controlador, usa foreach().

In [0]:
miRDD.foreach(lambda x: print(x))  # Aplica una operación sobre cada elemento del RDD


####6. first() y Uso Controlado de take()
Utiliza first() para obtener el primer elemento del RDD.
Si usas take(), elige un valor pequeño para evitar problemas de memoria.

In [0]:
primerElemento = miRDD.first()  # Obtiene el primer elemento del RDD


####7. glom()
glom() convierte cada partición en una lista, permitiendo trabajar con datos particionados sin traerlos todos al controlador.

In [0]:
particiones = miRDD.glom().collect()  # Cada partición se convierte en una lista


####8. limit(n)
Utiliza limit(n) para limitar el número de elementos en el RDD y evitar traer todo a la memoria.

In [0]:
primerosRDD = miRDD.limit(100)  # Limita el RDD a los primeros 100 elementos


> Funciones lambda
> 
> *Spark* hace un uso extensivo de las funciones *lambda de Python* para simplificar el código.
> Una función lambda no es más que una función anónima condensada en una sola línea:

In [0]:
# par1 y par2 son los parámetros de entrada, y expresión el código de la función
lambda par1, par2: expresión

Out[37]: <function __main__.<lambda>(par1, par2)>

``lambda:`` Es la palabra clave que indica que estás creando una función lambda.

``par1, par2:`` Son los parámetros de entrada de la función. Puedes tener cualquier número de parámetros separados por comas.

``expresión:`` Es el código de la función que realiza la operación. La expresión se evalúa y se devuelve automáticamente como resultado de la función.

> Por ejemplo, si tuviéramos una función que suma dos números, podríamos hacerla en Python de estas dos maneras:

In [0]:
def suma(x,y):
    return(x + y)

sumaL = lambda x,y : x + y

res = suma(3, 4)
resL = sumaL(3, 4)

A lo largo de los ejemplos de esta sesión y las siguientes nos iremos familiarizando con su uso

### Collect

Es una acción que se utiliza para recuperar todos los elementos de un RDD, DataFrame o Dataset desde los nodos distribuidos del clúster y traerlos al controlador (la máquina que ejecuta tu aplicación Spark).

Un fallo muy posible a la hora de mostrar los datos de un RDD es utilizar `rdd.foreach(print)` o `rdd.map(print)`.

En una única máquina, esta operación generaría la salida esperada mostrando todos los elementos del RDD. Sin embargo, al trabajar en un clúster, la salida a stdout la realizarían los diferentes nodos y no el nodo principal. Así pues, para mostrar todos los elementos de un RDD/DataFrame/Dataset hemos de emplear el método collect, el cual primero mostrará los RDD del nodo principal (driver node), y luego para cada nodo del cluster mostrará sus datos.

In [0]:
rdd.collect()

Out[39]: [0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100]

> **Out of memory**
> 
> Hay que tener mucho cuidado, ya que nos podemos quedar fácilmente sin memoria, ya que `collect` se trae los datos de todos los ejecutores a un único nodo, el que ésta ejecutando el código (driver).

Si sólo necesitamos mostrar unos pocos elementos, un enfoque más seguro es utilizar `take`:

In [0]:
rdd.take(100)

Out[40]: [0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

### Transformaciones

Las transformaciones son operaciones aplicadas a un RDD, DataFrame o Dataset que producen un nuevo conjunto de datos. Estas operaciones son perezosas (lazy), lo que significa que no se ejecutan inmediatamente, sino que solo se registran en un plan lógico de ejecución. Las transformaciones no procesan los datos directamente hasta que se llama a una acción.

En Spark, las estructuras de datos son inmutables, de manera que una vez creadas no se pueden modificar. Para poder modificar un RDD/DataFrame hace falta realizar una transformación, siendo el modo de expresar la lógica de negocio mediante Spark.

Todas las transformaciones en Spark se evalúan de manera perezosa (lazy evaluation), de manera que los resultados no se computan inmediatamente, sino que se retrasa el cálculo hasta que el valor sea necesario. Para ello, se van almacenando los pasos necesarios y se ejecutan únicamente cuando una acción requiere devolver un resultado al driver. Este diseño facilita un mejor rendimiento (por ejemplo, imagina que tras una operación map se realiza un reduce y en vez de devolver todo el conjunto de datos tras el map, sólo le enviamos al driver el resultado de la reducción).

####Explicación
La evaluación perezosa en Spark significa que las transformaciones no se ejecutan de inmediato. En su lugar, se aplazan hasta que una acción (como ``collect()`` o ``count()``) necesite un resultado. Esto permite que Spark optimice el proceso completo, combinando pasos y reduciendo el uso innecesario de recursos, lo cual mejora el rendimiento y eficiencia.

Así pues, las acciones provocan la evaluación de todas las transformaciones previas que se habían evaluado de forma perezosa y estaban a la espera.

Por defecto, cada transformación RDD/DataSet se puede recalcular cada vez que se ejecute una acción. Sin embargo, podemos persistir un RDD en memoria mediante los métodos persist (o cache), de manera que Spark mantendrá los datos para un posterior acceso más eficiente. También podemos persistir un RDD en disco o replicarlo en múltiples nodos.

### Tipos de transformaciones

Existen dos tipos de transformaciones, dependiendo de las dependencias entre las particiones de datos:

- Transformaciones **Narrow**: consisten en dependencias estrechas en las que cada partición de entrada contribuye a una única partición de salida.

> Explicar

En Spark, una transformación narrow (estrecha) es un tipo de transformación en la cual cada partición de los datos generados depende solo de una partición de los datos originales.

####Ejemplos Transformacion Narrow
``map()``: Toma cada elemento del RDD y lo transforma, pero la salida depende solo de la partición en la que se encuentra el elemento.

``filter()``: Selecciona elementos que cumplen con una condición, y cada partición resultante solo depende de los elementos originales en la misma partición.

- Transformaciones **Wide**: consisten en dependencias anchas de manera que varias particiones de entrada contribuyen a muchas otras particiones de salida, es decir, cada partición de salida depende de diferentes particiones de entrada. Este proceso también se conoce como shuffle, ya que Spark baraja los datos entre las particiones del clúster.

> Explicar

Una transformación wide (amplia) en Spark es una transformación donde cada partición del resultado depende de múltiples particiones del RDD original. Esto significa que Spark necesita reorganizar los datos y moverlos entre nodos para realizar la operación.

###Ejemplos transformacion Wide
``groupByKey()``: Agrupa los elementos por clave y necesita combinar datos que pueden estar en diferentes particiones.

``reduceByKey()``: También requiere combinar valores de diferentes particiones con la misma clave.


![7](https://community.cloud.databricks.com/files/Notebook/7.png)

Con las transformaciones narrow, Spark realiza un pipeline de las dependencias, de manera que si especificamos múltiples filtros sobre DataFrames/RDD, se realizarán todos en memoria.

Esto no sucede con las transformaciones wide, ya que al realizar un shuffle los resultados se persisten en disco.

El shuffle ocurre cuando Spark necesita repartir los datos de forma que ciertas claves o valores relacionados estén en la misma partición. Esto es necesario en transformaciones como groupByKey, reduceByKey, join, etc., donde los cálculos dependen de combinar datos relacionados que podrían estar en diferentes particiones.

> Cuidado con shuffle
> Las operaciones shuffle son computacionalmente caras, ya que implican E/S en disco, serialización de datos y E/S en red. Para organizar los datos previos al shuffle, Spark genera un conjunto de tareas (tareas map para organizar los datos, y reduce para agregar los resultados).
> Internamente, el resultado de las tareas map se mantienen en memoria hasta que no caben. Entonces, se ordenan en la partición destino y se persisten en un único archivo. En la fase de reducción, las tareas leen los bloques ordenados que son relevantes.
> Las operaciones reduceByKey y aggregateByKey son de las que más memoria consumen, al tener que crear las estructuras de datos para organizar los registros en las tareas de map, y luego generar los resultados agregados en la de reduce. Si los datos no caben en memoria, Spark los lleva a disco, incurriendo en operaciones adicionales de E/S en disco y del recolector de basura.

A continuación veremos las diferentes transformaciones que podemos realizar con Spark.

### Transformaciones Narrow

Para los siguientes ejemplo, utilizaremos el siguiente fichero de `empleados.txt` que contiene para cada empleado los siguiente cinco campos:

- el nombre

- un array de ciudades donde trabaja separadas por coma

- una estructura compuesta del sexo y la edad separadas de nuevo por comas

- un array de mapas con sus destrezas y calificación

- y finalmente un array de cargos y su consiguiente array de destrezas

### Map

 Es una transformación que aplica una función específica a cada elemento de un RDD, y devuelve un nuevo conjunto de datos donde cada elemento de salida es el resultado de aplicar la función a su correspondiente elemento de entrada. Es una de las transformaciones estrechas (narrow transformations), lo que significa que cada partición de salida depende de una única partición de entrada. Esto lo hace más eficiente porque no requiere un shuffle (reorganización de datos entre particiones).

La transformación [map](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html) aplica la función recibida a cada elemento del RDD, de manera que vamos a poder añadir una nueva columna, modificar una existente, etc...

Por ejemplo, si la entrada es un RDD que contiene `[1, 2, 3, 4, 5]`, al hacer `rdd.map(lambda x: x*2)` obtendríamos un nuevo RDD cuyos valores contiene su doble, es decir, `[2, 4, 6, 8, 10]`:

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
resultRDD = rdd.map(lambda x: x*2)
resultRDD.collect()          # [2, 4, 6, 8, 10]

Out[41]: [2, 4, 6, 8, 10]

> Explicar código de arriba

Crear un RDD con parallelize():


``rdd = sc.parallelize([1, 2, 3, 4, 5])``

``sc.parallelize()``: Este método crea un RDD (Resilient Distributed Dataset) a partir de una lista.

``[1, 2, 3, 4, 5]:`` Es la lista de datos que se convierte en un RDD.

``rdd``: Es el nombre del RDD que contiene los elementos [1, 2, 3, 4, 5].

(No hacer los codigos relativos a Hadoop o HDFS) Mediante la función textFile podemos cargar un archivo. Supongamos que tenemos cargado en Hadoop el archivo empleados.txt:

rddLocal = sc.textFile("empleados.txt")
rdd = sc.textFile("hdfs://iabd-virtualbox:9000/user/iabd/datos/empleados.txt") 
rdd.count()                 # 4 - cantidad de líneas
resultRDD = rdd.map(len)    # obtenemos la cantidad de caracteres de cada línea
resultRDD.collect()         # [61, 52, 60, 50]

Si sobre el fichero de empleados, realizamos un split para separar los campos haciendo uso de map mediante una función lambda:

resultMap = rddLocal.map(lambda x: x.split("|"))
resultMap.collect()

Obtenemos una lista de listas por cada fila del fichero, cada lista conteniendo los cincos campos de cada empleado:

[['Michael',
  'Montreal,Toronto',
  'Male,30',
  'DB:80',
  'Product:Developer\x04Lead'],
 ['Will', 'Montreal', 'Male,35', 'Perl:85', 'Product:Lead,Test:Lead'],
 ['Shelley', 'New York', 'Female,27', 'Python:80', 'Test:Lead,COE:Architect'],
 ['Lucy', 'Vancouver', 'Female,57', 'Sales:89,HR:94', 'Sales:Lead']]

### FlatMap

Es una transformación que aplica una función a cada elemento de un RDD, y genera cero, uno o más elementos por cada entrada. A diferencia de map, que siempre genera exactamente un elemento por entrada, flatMap "aplana" los resultados en un solo conjunto de datos. Es particularmente útil cuando se necesita dividir, expandir o descomponer los elementos de un conjunto de datos en varios elementos.

La transformación `flatMap` es muy similar a la anterior, pero en vez de devolver una lista con un elemento por cada entrada, devuelve una única lista deshaciendo las colecciones en elementos individuales:

In [0]:
rdd = sc.textFile("/FileStore/Notebook/empleados.txt") 
resultFM = rdd.flatMap(lambda x: x.split("|"))
resultFM.collect()
# Obtendríamos cada atributo separado y todos dentro de la misma lista

Out[43]: ['Michael',
 'Montreal,Toronto',
 'Male,30',
 'DB:80',
 'Product:Developer\x04Lead',
 'Will',
 'Montreal',
 'Male,35',
 'Perl:85',
 'Product:Lead,Test:Lead',
 'Shelley',
 'New York',
 'Female,27',
 'Python:80',
 'Test:Lead,COE:Architect',
 'Lucy',
 'Vancouver',
 'Female,57',
 'Sales:89,HR:94',
 'Sales:Lead']

### Filter

Es una transformación que se utiliza para filtrar elementos de un RDD según una condición específica. Solo los elementos que cumplen la condición se incluyen en el resultado, mientras que los demás se descartan.

Es una transformación estrecha (narrow), lo que significa que los elementos de salida de cada partición dependen únicamente de los elementos de entrada de la misma partición. Esto lo hace eficiente porque no requiere un shuffle (reorganización de datos entre nodos).

Permite filtrar los elementos que cumplen una condición mediante `filter`:

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
resultRDD = rdd.filter(lambda x: x%2==0)
resultRDD.collect()     # [2, 4]

Out[44]: [2, 4]

También podemos anidar diferentes transformaciones. Para este ejemplo, vamos a crear tuplas formadas por un número y su cuadrado, y luego quitar los que no coincide el número con su cuadrado (sólo coinciden el 0 y el 1), y luego aplanarlo en una lista:

In [0]:
rdd10 = sc.parallelize(range(10+1))
rddPares = rdd10.map(lambda x: (x, x**2)).filter(lambda x: (x[0] != x[1])).flatMap(lambda x: x)
rddPares.collect()      # [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

Out[45]: [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

> Explicar el código de arriba

Crear RDD del rango 0-10:

``rdd10 = sc.parallelize(range(10 + 1))``

rdd10 contiene los números del 0 al 10 (incluyendo 10).
Transformaciones encadenadas:

``rddPares = rdd10.map(lambda x: (x, x**2)).filter(lambda x: (x[0] != x[1])).flatMap(lambda x: x)``

``map(lambda x: (x, x**2))``: Convierte cada número en un par (x, x²) (por ejemplo, (2, 4)).

``filter(lambda x: (x[0] != x[1])):`` Filtra los pares donde el número no sea igual a su cuadrado (descarta el (0, 0) y (1, 1)).

``flatMap(lambda x: x)``: Aplana los pares (x, x²) en elementos individuales.
Recolectar los resultados:


``rddPares.collect()  # [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]``

Devuelve una lista de los números y sus cuadrados que no fueron iguales (2, 4, 3, 9, ..., 10, 100).

Veamos otro ejemplo. Retomamos los datos de los empleados y si queremos filtrar los empleados que son hombres, primero separamos por las | y nos quedamos con el tercer elemento que contiene el sexo y la edad. A continuación, separamos por la coma para quedarnos en el sexo en la posición 0 y la edad en el 1, y comparamos con el valor deseado:

In [0]:

rdd = sc.textFile("/FileStore/Notebook/empleados.txt") 
# Filtrar registros donde el campo de género contiene "Male"
hombres = rdd.filter(lambda x: len(x.split("|")) > 2 and x.split("|")[2].split(",")[0] == "Male")

# Mostrar los resultados
print(hombres.collect())

['Michael|Montreal,Toronto|Male,30|DB:80|Product:Developer\x04Lead', 'Will|Montreal|Male,35|Perl:85|Product:Lead,Test:Lead']


> Explicar el codigo de arriba

Leer el archivo de texto:

``rdd = sc.textFile("/FileStore/Notebook/empleados.txt")``

Crea un RDD a partir del archivo empleados.txt. Cada línea del archivo se convierte en un elemento del RDD.
Filtrar registros donde el campo de género es "Male":


``hombres = rdd.filter(lambda x: len(x.split("|")) > 2 and x.split("|")[2].split(",")[0] == "Male")``

Divide cada línea usando "|" como separador para obtener diferentes campos.
Filtra las líneas donde el tercer campo es "Male".
Recolectar y mostrar resultados:


``print(hombres.collect())``

Recolecta todos los registros filtrados en una lista y los imprime.

Obteniendo:

['Michael|Montreal,Toronto|Male,30|DB:80|Product:Developer\x04Lead',
 'Will|Montreal|Male,35|Perl:85|Product:Lead,Test:Lead']

### Distinct

Si utilizamos [distinct](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.distinct.html) eliminaremos los elementos repetidos:

### Trabajando con conjuntos

**Union**

Mediante [union](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.union.html) unimos dos RDD en uno:

In [0]:
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([5,6,7,8])
resultRDD = rdd1.union(rdd2)
resultRDD.collect()     # [1, 2, 3, 4, 5, 6, 7, 8]

Out[48]: [1, 2, 3, 4, 5, 6, 7, 8]

**Intersection**

Mediante [intersection](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.intersection.html), obtendremos los elementos que tengan en común:

**Subtract**

Mediante [subtract](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.subtract.html), obtendremos los elementos propios que no estén en el RDD recibido:

In [0]:
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([3,4,5,6])
resultRDD = rdd1.subtract(rdd2)
resultRDD.collect()     # [1, 2]

Out[49]: [1, 2]

### Actividad:

Si tenemos dos RDD (A y B):

In [0]:
rddA = sc.parallelize([1,2,3,4])
rddB = sc.parallelize([3,4,5,6])

¿Cómo conseguimos los elementos que están en A y no B y los de B que no están en A? (es decir [1, 2, 5, 6])):

In [0]:
# Escriba la respuesta aqui
# Elementos que están en A y no en B
diferencia_A_B = rddA.subtract(rddB)

# Elementos que están en B y no en A
diferencia_B_A = rddB.subtract(rddA)

# Unir ambas diferencias para obtener los elementos únicos en A y en B
resultado = diferencia_A_B.union(diferencia_B_A)

# Recolectar los resultados para verlos
print(resultado.collect())  # Salida: [1, 2, 5, 6]


[1, 2, 5, 6]


### RDD de Pares

Una técnica muy común a la hora de trabajar con RDD es hacerlo con elementos que tienen el formato (`clave, valor`), pudiendo las claves y los valores ser de cualquier tipo.

In [0]:
listaTuplas = [(1,'a'), (2,'b'), (3,'c'), (4,'d')]
rddTuplas= sc.parallelize(listaTuplas)

Sobre estos RDD podemos realizar algoritmos MapReduce para muchas funciones de procesamiento de datos, como es la agrupación, ordenación, join, count, etc...

Para generar un RDD de pares, además de crearlo nosotros a partir de una lista, podemos emplear las siguientes operaciones:

- [zip](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.zip.html): une dos RDDs del mismo tamaño:

In [0]:
lista1 = ['a','b','c','e','f','g','h']
lista2 = [4, 5, 6, 7, 8, 9, 10]
rddZip = sc.parallelize(lista1).zip(sc.parallelize(lista2)).collect()
# [('a', 4), ('b', 5), ('c', 6), ('e', 7), ('f', 8), ('g', 9), ('h', 10)]

In [0]:
rddZipSecuencia = sc.parallelize(zip(lista1,range(len(lista1)))) # usando el tamaño de la lista
# [('a', 0), ('b', 1), ('c', 2), ('e', 3), ('f', 4), ('g', 5), ('h', 6)]

- `map`: asignando a cada elemento un valor o cálculo sobre él mismo:

In [0]:
lista  = ['Hola', 'Adiós', 'Hasta luego']
rddMap = sc.parallelize(lista).map(lambda x: (x, len(x)))
# [('Hola', 4), ('Adiós', 5), ('Hasta luego', 11)]

- `keyBy`: permite crear las claves a partir de cada elemento:

In [0]:
rddKeyBy = sc.parallelize(lista).keyBy(lambda x: x[0])  # creamos una clave con la primera letra
# [('H', 'Hola'), ('A', 'Adiós'), ('H', 'Hasta luego')]

### Actividad:

A partir de la lista `"Perro Gato Loro Pez León Tortuga Gallina"`

1. Crea un RDD a partir de esta lista
2. Convierte el RDD normal en un RDD de pares cuya clave sea la primera letra del animal
3. Crea otro RDD de pares pero poniendo como clave un número incremental
4. ¿Y si queremos que el índice incremental empiece en 100?

#### Solucion

In [0]:
# Coloca tus respuestas aqui

# Crear RDD a partir de la lista de animales
animales = ["Perro", "Gato", "Loro", "Pez", "León", "Tortuga", "Gallina"]
rdd = sc.parallelize(animales)

# Convertir el RDD en un RDD de pares 
rddPares = rdd.map(lambda x: (x[0], x))
print(rddPares.collect())

# Crear un RDD de pares con un índice incremental como clave
rddConIndice = rdd.zipWithIndex().map(lambda x: (x[1], x[0]))
print(rddConIndice.collect())

# Crear un RDD de pares con un índice incremental empezando en 100
rddConIndice100 = rdd.zipWithIndex().map(lambda x: (x[1] + 100, x[0]))
print(rddConIndice100.collect())






[('P', 'Perro'), ('G', 'Gato'), ('L', 'Loro'), ('P', 'Pez'), ('L', 'León'), ('T', 'Tortuga'), ('G', 'Gallina')]
[(0, 'Perro'), (1, 'Gato'), (2, 'Loro'), (3, 'Pez'), (4, 'León'), (5, 'Tortuga'), (6, 'Gallina')]
[(100, 'Perro'), (101, 'Gato'), (102, 'Loro'), (103, 'Pez'), (104, 'León'), (105, 'Tortuga'), (106, 'Gallina')]


Sobre los RDD de pares, podemos realizar las siguientes transformaciones:

- [`keys`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.keys.html): devuelve las claves
- [`values`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.values.html): devuelve los valores
- [`mapValues`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.mapValues.html): Aplica la función sobre los valores
- [`flatMapValues`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.flatMapValues.html) Aplica la función sobre los valores y los aplana.
  

A continuación se muestra un fragmento de código para poner en práctica las transformaciones comentadas:

In [0]:
listaTuplas = [('a',1), ('z',3), ('b',4), ('c',3), ('a',4)]
rddTuplas = sc.parallelize(listaTuplas)

claves = rddTuplas.keys()       # ['a', 'z', 'b', 'c', 'a']
valores = rddTuplas.values()    # [1, 3, 4, 3, 4]

rddMapValues = rddTuplas.mapValues(lambda x: (x,x*2))
# [('a', (1, 2)), ('z', (3, 6)), ('b', (4, 8)), ('c', (3, 6)), ('a', (4, 8))]
rddFMV = rddTuplas.flatMapValues(lambda x: (x,x*2))
# [('a', 1),
#  ('a', 2),
#  ('z', 3),
#  ('z', 6),
#  ('b', 4),
# ...

### Transformaciones Wide

Las siguientes transformaciones, además de trabajar con RDD de pares, mezclan los datos de las particiones mediante el shuffle de los elementos.

Para los siguientes ejemplos, utilizaremos un fichero de ventas [`pdi_sales.csv`](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EdbX1Ds2BShClAxQaHHZYyYBwu5DY1Li3jvXLN_Z7KbhHg?e=Vj3xFq) (versión extendida) / [`pdi_sales_small.csv`](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EdbX1Ds2BShClAxQaHHZYyYBwu5DY1Li3jvXLN_Z7KbhHg?e=Vj3xFq) (versión reducida) el cual tiene el siguiente formato:

ProductID;Date;Zip;Units;Revenue;Country
725;1/15/1999;41540          ;1;115.5;Germany
787;6/6/2002;41540          ;1;314.9;Germany
...

### ReduceByKey

Mediante la transformación [`reducedByKey`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html) los datos se calculan utilizando una función de reducción a partir de la clave combinando en la misma máquina las parejas con la misma clave antes de que los datos se barajen.

Vamos a comenzar con un ejemplo sencillo, contando cuantas ventas se han realizado en cada país, o lo que es lo mismo, las veces que aparece cada palabra en el fichero:

In [0]:
rdd = sc.textFile("dbfs:/FileStore/Notebook/pdi_sales_small.csv")
# Recogemos el país y las unidades de las ventas
parPais1 = rdd.map(lambda x: (x.split(";")[-1].strip(), 1))
# parPais1.collect()
# [('Country', 1),
#  ('Germany', 1),
#  ('Germany', 1),
#  ('Germany', 1), ...

Hemos creado un RDD de pares compuesto por el nombre del país y el número uno, para luego en la fase de reducción sumar estos valores. Pero si nos fijamos, el archivo csv contiene el encabezado con los datos, el cual debemos quitar:

In [0]:
header = parPais1.first()
parPais1SinHeader = parPais1.filter(lambda linea: linea != header)
# parPais1SinHeader.collect()
# [('Germany', 1),
#  ('Germany', 1),
#  ('Germany', 1), ...

Y finalmente, ya podemos reducir por la clave:

In [0]:
paisesTotal = parPais1SinHeader.reduceByKey(lambda a,b: a+b)
# paisesTotal.collect()
# [('Mexico', 30060), ('France', 30060), ('Germany', 30059), ('Canada', 30060)]

> Funciones lambda en reduce
> Al aplicar una transformación de tipo reduce, la función lambda recibirá dos parámetros, siendo el primero el valor acumulado y el segundo el valor del elemento a operar.

Veamos otro ejemplo, en este caso vamos a calcular el total de unidades vendidas por país, de manera que vamos a coger el nombre del país (Country) y las unidades (Units) vendidas:

In [0]:
rdd = sc.textFile("dbfs:/FileStore/Notebook/pdi_sales_small.csv")
# Recogemos el país y las unidades de las ventas
paisesUnidades = rdd.map(lambda x: (x.split(";")[-1].strip(), x.split(";")[3]))
# Le quitamos el encabezado
header = paisesUnidades.first()
paisesUnidadesSinHeader = paisesUnidades.filter(lambda linea: linea != header)
# Pasamos las unidades a un número entero
paisesUnidadesInt = paisesUnidadesSinHeader.map(lambda x: (x[0], int(x[1])))
# Reducimos por el país y sumamos las unidades
paisesTotalUnidades = paisesUnidadesInt.reduceByKey(lambda a,b: a+b)
paisesTotalUnidades.collect()

Out[68]: [('Mexico', 31095), ('France', 31739), ('Germany', 31746), ('Canada', 31148)]

### Actividad:

Dada la siguiente lista de compra:

In [0]:
lista = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]

**Calcula**

1. El total que se ha gastado por cada producto  
2. Cuánto es lo máximo que se ha pagado por cada producto

In [0]:
# Crear RDD
lista = [('pan', 3), ('agua', 2), ('azúcar', 1), ('leche', 2), ('pan', 1), ('cereales', 3), ('agua', 0.5), ('leche', 2), ('filetes', 5)]
rdd = sc.parallelize(lista)

#Calcular el total gastado
totalPorProducto = rdd.reduceByKey(lambda a, b: a + b)
print(totalPorProducto.collect())



[('agua', 2.5), ('pan', 4), ('leche', 4), ('cereales', 3), ('filetes', 5), ('azúcar', 1)]


### GroupByKey

Permite agrupar los datos a partir de una clave, repartiendo los resultados (shuffle) entre todos los nodos:

In [0]:
rdd = sc.textFile("dbfs:/FileStore/Notebook/pdi_sales_small.csv")
# Creamos un RDD de pares con el nombre del país como clave, y una lista con los valores
ventas = rdd.map(lambda x: (x.split(";")[-1].strip(), x.split(";")))
# Quitamos el primer elemento que es el encabezado del CSV
header = paisesUnidades.first()
paisesUnidadesSinHeader = paisesUnidades.filter(lambda linea: linea != header)
# Agrupamos las ventas por nombre del país
paisesAgrupados = ventas.groupByKey()
paisesAgrupados.collect()
# Obtendremos para cada pais, un iterable con todos sus datos:

Out[76]: [('Country', <pyspark.resultiterable.ResultIterable at 0x7f5441b91940>),
 ('Mexico', <pyspark.resultiterable.ResultIterable at 0x7f5441b916d0>),
 ('France', <pyspark.resultiterable.ResultIterable at 0x7f5441d4c070>),
 ('Germany', <pyspark.resultiterable.ResultIterable at 0x7f545cf07f70>),
 ('Canada', <pyspark.resultiterable.ResultIterable at 0x7f545c7ba5e0>)]

Podemos transformar los iterables a una lista:

In [0]:
paisesAgrupadosLista = paisesAgrupados.map(lambda x: (x[0], list(x[1])))
paisesAgrupadosLista.collect()

Out[77]: [('Country', [['ProductID', 'Date', 'Zip', 'Units', 'Revenue', 'Country']]),
 ('Mexico',
  [['2235', '1/15/1999', '8650           ', '1', '65.6', 'Mexico '],
   ['837', '2/15/1999', '8650           ', '1', '840', 'Mexico '],
   ['491', '2/15/1999', '8650           ', '1', '815.3', 'Mexico '],
   ['426', '5/31/2002', '8650           ', '1', '843.1', 'Mexico '],
   ['400', '6/6/2002', '8650           ', '1', '734.7', 'Mexico '],
   ['1131', '2/15/1999', '8650           ', '1', '419.9', 'Mexico '],
   ['2277', '3/15/1999', '8650           ', '1', '251.9', 'Mexico '],
   ['491', '3/15/1999', '8650           ', '1', '815.3', 'Mexico '],
   ['467', '6/8/2002', '8650           ', '1', '900.6', 'Mexico '],
   ['565', '4/12/2003', '2000           ', '1', '890.7', 'Mexico '],
   ['604', '4/12/2003', '2000           ', '1', '494.8', 'Mexico '],
   ['940', '3/15/1999', '2000           ', '1', '687.7', 'Mexico '],
   ['2277', '1/15/1999', '8640           ', '1', '251.9', 'Mexico '],
   ['7

### Actividad 

- día 1: pan 3€, agua 2€, azúcar 1€, leche 2€, pan 4€
- día 2: pan 1€, cereales 3€, agua 0.5€, leche 2€, filetes 5€
- día 3: filetes 2€, cereales 1€

Dada la siguiente lista de compra:

In [0]:
dia1 = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',4)]
dia2 = [('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
dia3 = [('filetes',2), ('cereales',1)]

1. ¿Cómo obtenemos lo que hemos gastado en cada producto?
2. ¿Y el gasto medio que hemos realizado en cada uno de ellos?

In [0]:
# Respuestas aqui
# Crear RDDs a partir de las listas de cada dia
# Crear RDDs para cada día
rddDia1 = sc.parallelize(dia1)
rddDia2 = sc.parallelize(dia2)
rddDia3 = sc.parallelize(dia3)

# Unir los RDDs de los tres días
rddUnion = rddDia1.union(rddDia2).union(rddDia3)

# Calcular el total gastado por cada producto
totalPorProducto = rddUnion.reduceByKey(lambda a, b: a + b)
print(totalPorProducto.collect())

[('agua', 2.5), ('leche', 4), ('pan', 8), ('cereales', 4), ('filetes', 7), ('azúcar', 1)]


> Mejor `reduceByKey` que `groupByKey`  
> Si el tipo de operación a realizar es posible mediante una operación de reduce, su rendimiento será una solución más eficiente. Más información en el artículo [Avoid Group By](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/best_practices/prefer_reducebykey_over_groupbykey.html)

### SortByKey

`sortByKey` permite ordenar los datos a partir de una clave. Los pares de la misma máquina se ordenan primero por la misma clave, y luego los datos de las diferentes particiones se barajan.  
Para ello crearemos una tupla, siendo el primer elemento un valor numérico por el cual ordenar, y el segundo el dato asociado.  
Vamos a partir del ejemplo anterior para ordenar los paises por la cantidad de ventas:  


In [0]:
# Ejemplo anterior
rdd = sc.textFile("dbfs:/FileStore/Notebook/pdi_sales.csv")
paisesUnidades = rdd.map(lambda x: (x.split(";")[-1].strip(), x.split(";")[3]))
header = paisesUnidades.first()
paisesUnidadesSinHeader = paisesUnidades.filter(lambda linea: linea != header)
paisesTotalUnidades = paisesUnidadesSinHeader.reduceByKey(lambda a,b: int(a)+int(b))

In [0]:
# Le damos la vuelta a la lista
unidadesPaises = paisesTotalUnidades.map(lambda x: (x[1],x[0]))
unidadesPaises.collect()

Out[84]: [(223463, 'Mexico'),
 (327730, 'France'),
 (244265, 'Germany'),
 (77609, 'Canada')]

Y a continuación los ordenamos:

In [0]:
unidadesPaisesOrdenadas = unidadesPaises.sortByKey()
unidadesPaisesOrdenadas.collect()

Out[85]: [(77609, 'Canada'),
 (223463, 'Mexico'),
 (244265, 'Germany'),
 (327730, 'France')]

Y comprobamos el resultado:

In [0]:
[(77609, 'Canada'),
 (223463, 'Mexico'),
 (244265, 'Germany'),
 (327730, 'France')]

Out[86]: [(77609, 'Canada'),
 (223463, 'Mexico'),
 (244265, 'Germany'),
 (327730, 'France')]

Si quisiéramos obtener los datos en orden descendente, le pasamos False a la transformación:

In [0]:
unidadesPaisesOrdenadasDesc = unidadesPaises.sortByKey(False)

### SortBy

Mediante `sortBy` podemos ordenar los datos indicando nosotros la función de ordenación:

In [0]:
paisesTotalUnidades.sortBy(lambda x: x[1]).collect()

Out[87]: [('Canada', 77609),
 ('Mexico', 223463),
 ('Germany', 244265),
 ('France', 327730)]

Si queremos ordenar descendentemente, le pasamos un segundo parámetro con valor `False` (indica si la ordenación es ascendente):

In [0]:
paisesTotalUnidades.sortBy(lambda x: x[1], False).collect()

Out[88]: [('France', 327730),
 ('Germany', 244265),
 ('Mexico', 223463),
 ('Canada', 77609)]

> **Join**  
> Aunque los RDD permitan realizar operaciones join, realmente este tipo de operaciones se realizan mediante DataFrames, por lo que omitimos su explicación en esta sesión y la dejamos para la siguiente.

### Particiones

Spark organiza los datos en particiones, considerándolas divisiones lógicas de los datos entre los nodos del clúster. Por ejemplo, si el almacenamiento se realiza en HDFS, cada partición se asigna a un bloque.

Cada una de las particiones va a llevar asociada una tarea de ejecución, de manera que a más particiones, mayor paralelización del proceso.

Veamos con código como podemos trabajar con las particiones:

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5])
rdd.getNumPartitions() # 4

Out[89]: 8

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)
rdd.getNumPartitions() # 2

Out[90]: 2

In [0]:
rddE = sc.textFile("dbfs:/FileStore/Notebook/empleados.txt")
rddE.getNumPartitions() # 2

Out[92]: 2

In [0]:
rddE = sc.textFile("dbfs:/FileStore/Notebook/empleados.txt", 3)
rddE.getNumPartitions() # 3

Out[93]: 3

La mayoría de operaciones / transformaciones / acciones que trabajan con los datos admiten un parámetro extra indicando la cantidad de particiones con las que queremos trabajar.

### MapPartitions

A diferencia de la transformación map que se invoca por cada elemento del RDD/DataSet, mapPartitions se llama por cada partición.

La función que recibe como parámetro recogerá como entrada un iterador con los elementos de cada partición:

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)

In [0]:
def f(iterator): yield sum(iterator)
resultadoRdd = rdd.mapPartitions(f)
resultadoRdd.collect()  # [6, 15]

Out[95]: [6, 15]

In [0]:
resultadoRdd2 = rdd.mapPartitions(lambda iterator: [list(iterator)])
resultadoRdd2.collect() # [[1, 1, 2, 2], [3, 3, 4, 5]]

Out[96]: [[1, 1, 2, 2], [3, 3, 4, 5]]

En el ejemplo, ha dividido los datos en dos particiones, la primera con [1, 1, 2, 2] y la otra con [3, 3, 4, 5], y de ahí el resultado de sumar sus elementos es [6, 15].

### mapPartitionsWithIndex

De forma similar al caso anterior, pero ahora [mapPartitionsWithIndex](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.mapPartitionsWithIndex.html) recibe una función cuyos parámetros son el índice de la partición y el iterador con los datos de la misma:

In [0]:
def mpwi(indice, iterador):
    return [(indice, list(iterador))]

resultadoRdd = rdd.mapPartitionsWithIndex(mpwi)
resultadoRdd.collect()
# [(0, [1, 1, 2, 2]), (1, [3, 3, 4, 5])]

Out[97]: [(0, [1, 1, 2, 2]), (1, [3, 3, 4, 5])]

### Modificando las particiones

Podemos modificar la cantidad de particiones mediante dos transformaciones *wide*: `coalesce` y `repartition`.

Mediante [`coalesce`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.coalesce.html) podemos obtener un nuevo RDD con la cantidad de particiones a reducir:

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
rdd.getNumPartitions() # 3

Out[98]: 3

In [0]:
rdd1p = rdd.coalesce(1)
rdd1p.getNumPartitions() # 2

Out[99]: 1

En cambio, mediante repartition podemos obtener un nuevo RDD con la cantidad exacta de particiones deseadas (al reducir las particiones, repartition realiza un shuffle para redistribuir los datos, por lo tanto, si queremos reducir la cantidad de particiones, es más eficiente utilizar *coalesce*):

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
rdd.getNumPartitions() # 3

Out[100]: 3

In [0]:
rdd2p = rdd.repartition(2)
rdd2p.getNumPartitions() # 2

Out[101]: 2

### Actividades

En las siguientes actividades vamos a familiarizarnos con el uso de Spark con RDD y las diferentes acciones y transformaciones disponibles.

**1.** A partir de la lista siguiente ['Alicante','Elche','Valencia','Madrid','Barcelona','Bilbao','Sevilla']:

In [0]:
# Crear RDD a partir de la lista de ciudades
ciudades = ['Alicante', 'Elche', 'Valencia', 'Madrid', 'Barcelona', 'Bilbao', 'Sevilla']
rddCiudades = sc.parallelize(ciudades)


a. Almacena sólo las ciudades que tengan la letra e en su nombre y muéstralas.

In [0]:
ciudadesConE = rddCiudades.filter(lambda ciudad: 'e' in ciudad)
print(ciudadesConE.collect())


['Alicante', 'Elche', 'Valencia', 'Barcelona', 'Sevilla']


En Python, para saber si una cadena contiene una letra puedes usar el operador in:

In [0]:
nombre = "Aitor Medrano"
print("i" in nombre) # True
print("u" in nombre) # False
print("Med" in nombre) # True
print("ai" in nombre) # False (case sensitive)

True
False
True
False


b. Muestra las ciudades que tienen la letra e y el número de veces que aparece en cada nombre. Por ejemplo ('Elche', 2).

In [0]:
ciudadesConEContadas = ciudadesConE.map(lambda ciudad: (ciudad, ciudad.count('e')))
print(ciudadesConEContadas.collect())


[('Alicante', 1), ('Elche', 1), ('Valencia', 1), ('Barcelona', 1), ('Sevilla', 1)]


c. Averigua las ciudades que solo tengan una única e.

In [0]:
ciudadesUnaE = ciudadesConEContadas.filter(lambda x: x[1] == 1)
print(ciudadesUnaE.collect())


[('Alicante', 1), ('Elche', 1), ('Valencia', 1), ('Barcelona', 1), ('Sevilla', 1)]


d. Nos han enviado una nueva lista pero no han separado bien las ciudades. Reorganiza la lista y colócalas correctamente, y cuenta las apariciones de la letra e de cada ciudad. ciudades_mal = [['Alicante.Elche','Valencia','Madrid.Barcelona','Bilbao.Sevilla'],['Murcia','San Sebastián','Melilla.Aspe']]

In [0]:
# Nueva lista mal formada
ciudades_mal = [['Alicante.Elche', 'Valencia', 'Madrid.Barcelona', 'Bilbao.Sevilla'], ['Murcia', 'San Sebastián', 'Melilla.Aspe']]

# Crear un RDD a partir de la lista mal formada
rddCiudadesMal = sc.parallelize(ciudades_mal)

# Reorganizar la lista: Separar cada ciudad usando "." como delimitador y aplanar el resultado
rddCiudadesBien = rddCiudadesMal.flatMap(lambda lista: [ciudad for sub in lista for ciudad in sub.split('.')])

# Contar las apariciones de la letra "e" en cada ciudad
ciudadesConEContadasBien = rddCiudadesBien.map(lambda ciudad: (ciudad, ciudad.count('e')))
print(ciudadesConEContadasBien.collect())


[('Alicante', 1), ('Elche', 1), ('Valencia', 1), ('Madrid', 0), ('Barcelona', 1), ('Bilbao', 0), ('Sevilla', 1), ('Murcia', 0), ('San Sebastián', 1), ('Melilla', 1), ('Aspe', 1)]


**2.** Dada una lista de elementos desordenados y algunos repetidos, devolver una muestra de 5 elementos, que estén en la lista, sin repetir y ordenados descendentemente.

In [0]:
lista = 4,6,34,7,9,2,3,4,4,21,4,6,8,9,7,8,5,4,3,22,34,56,98

In [0]:
rdd = sc.parallelize(lista)

# Quitar duplicados, tomar una muestra de 5 elementos y ordenarlos descendentemente
rddSinDuplicados = rdd.distinct()
muestraOrdenada = rddSinDuplicados.takeOrdered(5, key=lambda x: -x)
print(muestraOrdenada)  # [98, 56, 34, 22, 21]


[98, 56, 34, 22, 21]


a. Selecciona el elemento mayor de la lista resultante.  
b. Muestra los dos elementos menores.

In [0]:
# a.
mayorElemento = max(muestraOrdenada)
print(mayorElemento)  # 98


98


In [0]:
# b.
menoresElementos = sorted(muestraOrdenada)[0:2]
print(menoresElementos)  # [21, 22]


[21, 22]


**3.** A partir de las siguientes listas:

- Inglés: hello, table, angel, cat, dog, animal, chocolate, dark, doctor, hospital, computer.
- Español: hola, mesa, angel, gato, perro, animal, chocolate, oscuro, doctor, hospital, ordenador.

In [0]:
ingles = ['hello', 'table', 'angel', 'cat', 'dog', 'animal', 'chocolate', 'dark', 'doctor', 'hospital', 'computer']
espanol = ['hola', 'mesa', 'angel', 'gato', 'perro', 'animal', 'chocolate', 'oscuro', 'doctor', 'hospital', 'ordenador']


Una vez creado un RDD con tuplas de palabras y su traducción (puedes usar zip para unir dos listas):

In [0]:
rddInglesEspanol = sc.parallelize(list(zip(ingles, espanol)))


[('hello', 'hola'),
 ('table', 'mesa'),
 ('angel', 'angel'),
 ('cat', 'gato')...

Averigua:  

a. Palabras que se escriben igual en inglés y en español.  
b. Palabras que en español son distintas que en inglés.  
c. Obtén una única lista con las palabras en ambos idiomas que son distintas entre ellas (['hello', 'hola', 'table', ...).  
d. Haz dos grupos con todas las palabras, uno con las que empiezan por vocal y otro con las que empiecen por consonante.  

In [0]:
#a
palabrasIguales = rddInglesEspanol.filter(lambda x: x[0] == x[1])
print(palabrasIguales.collect())  # [('angel', 'angel'), ('animal', 'animal'), ('chocolate', 'chocolate'), ('doctor', 'doctor'), ('hospital', 'hospital')]




[('angel', 'angel'), ('animal', 'animal'), ('chocolate', 'chocolate'), ('doctor', 'doctor'), ('hospital', 'hospital')]


In [0]:
#b
palabrasDistintas = rddInglesEspanol.filter(lambda x: x[0] != x[1])
print(palabrasDistintas.collect())  # [('hello', 'hola'), ('table', 'mesa'), ('cat', 'gato'), ('dog', 'perro'), ('dark', 'oscuro'), ('computer', 'ordenador')]

[('hello', 'hola'), ('table', 'mesa'), ('cat', 'gato'), ('dog', 'perro'), ('dark', 'oscuro'), ('computer', 'ordenador')]


In [0]:
#c
palabrasDistintasLista = palabrasDistintas.flatMap(lambda x: [x[0], x[1]]).collect()
print(palabrasDistintasLista)  # ['hello', 'hola', 'table', 'mesa', 'cat', 'gato', 'dog', 'perro', 'dark', 'oscuro', 'computer', 'ordenador']

['hello', 'hola', 'table', 'mesa', 'cat', 'gato', 'dog', 'perro', 'dark', 'oscuro', 'computer', 'ordenador']


In [0]:
#d
todasLasPalabras = rddInglesEspanol.flatMap(lambda x: [x[0], x[1]])

vocales = todasLasPalabras.filter(lambda palabra: palabra[0].lower() in ['a', 'e', 'i', 'o', 'u'])
print(vocales.collect())  # ['angel', 'animal', 'oscuro', 'ordenador']
consonantes = todasLasPalabras.filter(lambda palabra: palabra[0].lower() not in ['a', 'e', 'i', 'o', 'u'])
print(consonantes.collect())  # ['hello', 'hola', 'table', 'mesa', 'cat', 'gato', 'dog', 'perro', 'chocolate', 'doctor', 'hospital', 'computer']

['angel', 'angel', 'animal', 'animal', 'oscuro', 'ordenador']
['hello', 'hola', 'table', 'mesa', 'cat', 'gato', 'dog', 'perro', 'chocolate', 'chocolate', 'dark', 'doctor', 'doctor', 'hospital', 'hospital', 'computer']


**4.** A partir del fichero de [El Quijote](https://tajamar365.sharepoint.com/:t:/s/3405-MasterIA2024-2025/ERkcxiK7gt5Fr2KDytLuTXYBI3LdxtCa6IWeNvOTHFxbYQ?e=hCkatQ):

In [0]:
# Crear un RDD a partir del fichero de "El Quijote"
rddQuijote = sc.textFile("dbfs:/FileStore/Notebook/el_quijote.txt")


a. Crear un RDD a partir del fichero y crea una lista con todas las palabras del documento.  
b. ¿Cuantas veces aparece la palabra Dulcinea (independientemente de si está en mayúsculas o minúsculas)? ¿Y Rocinante? (86 y 120 ocurrencias respectivamente)  
c. Devuelve una lista ordenada según el número de veces que sale cada palabra de más a menos (las primeras ocurrencias deben ser [('que', 10731), ('de', 9035), ('y', 8668), ('la', 5014), ...).  

In [0]:
#a
palabrasRDD = rddQuijote.flatMap(lambda linea: linea.split())


In [0]:
#b
# Convertir todas las palabras a minúsculas
palabrasMinusculasRDD = palabrasRDD.map(lambda palabra: palabra.lower())

# Contar cuántas veces aparece "dulcinea"
ocurrenciasDulcinea = palabrasMinusculasRDD.filter(lambda palabra: palabra == "dulcinea").count()
print(f"Ocurrencias de Dulcinea: {ocurrenciasDulcinea}")  # Salida esperada: 86

# Contar cuántas veces aparece "rocinante"
ocurrenciasRocinante = palabrasMinusculasRDD.filter(lambda palabra: palabra == "rocinante").count()
print(f"Ocurrencias de Rocinante: {ocurrenciasRocinante}")  # Salida esperada: 120


Ocurrencias de Dulcinea: 47
Ocurrencias de Rocinante: 43


In [0]:
#c
# Contar las apariciones de cada palabra
palabrasContadasRDD = palabrasMinusculasRDD.map(lambda palabra: (palabra, 1)).reduceByKey(lambda a, b: a + b)

# Ordenar las palabras según la frecuencia de más a menos
palabrasOrdenadas = palabrasContadasRDD.sortBy(lambda x: x[1], ascending=False)

# Recoger los resultados
resultado = palabrasOrdenadas.collect()
print(resultado[:10])  # Mostrar las 10 palabras más comunes


[('que', 10384), ('de', 9026), ('y', 8449), ('la', 5009), ('a', 4806), ('en', 4030), ('el', 3842), ('no', 2910), ('se', 2382), ('los', 2147)]


**5.** Dada una cadena que contiene una lista de nombres Juan, Jimena, Luis, Cristian, Laura, Lorena, Cristina, Jacobo, Jorge, una vez transformada la cadena en una lista y luego en un RDD:  
a. Agrúpalos según su inicial, de manera que tengamos tuplas formadas por la letra inicial y todos los nombres que comienzan por dicha letra:

In [0]:
# Cadena con los nombres
cadena = "Juan, Jimena, Luis, Cristian, Laura, Lorena, Cristina, Jacobo, Jorge"

# Transformar la cadena en una lista
lista_nombres = cadena.split(", ")

# Crear un RDD a partir de la lista
rddNombres = sc.parallelize(lista_nombres)


[('J', ['Juan', 'Jimena', 'Jacobo', 'Jorge']),
('L', ['Luis', 'Laura', 'Lorena']),
('C', ['Cristian', 'Cristina'])]

In [0]:
# Crear pares (inicial, nombre)
rddInicialNombre = rddNombres.map(lambda nombre: (nombre[0], nombre))

# Agrupar los nombres por inicial
agrupadosPorInicial = rddInicialNombre.groupByKey().mapValues(list)

# Mostrar los resultados
print(agrupadosPorInicial.collect())


[('J', ['Juan', 'Jimena', 'Jacobo', 'Jorge']), ('C', ['Cristian', 'Cristina']), ('L', ['Luis', 'Laura', 'Lorena'])]


b. De la lista original, obtén una muestra de 5 elementos sin repetir valores.

In [0]:
muestraSinRepetir = rddNombres.distinct().takeSample(False, 5)
print(muestraSinRepetir)


['Juan', 'Cristian', 'Cristina', 'Jacobo', 'Laura']


c. Devuelve una muestra de datos de aproximadamente la mitad de registros que la lista original con datos que pudieran llegar a repetirse.

In [0]:
muestraConRepeticion = rddNombres.sample(withReplacement=True, fraction=0.5)
print(muestraConRepeticion.collect())


['Juan', 'Luis', 'Cristian', 'Cristian', 'Lorena', 'Lorena', 'Lorena', 'Cristina']


**6.** En una red social sobre cine, tenemos un fichero [ratings.txt](https://tajamar365.sharepoint.com/:t:/s/3405-MasterIA2024-2025/EaYOt-hRpDBIl2DrlGnLx38Bqolu9xt7f4weY1We7jrkCQ?e=EwltPA) compuesta por el código de la película, el código del usuario, la calificación asignada y el timestamp de la votación con el siguiente formato:

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968

Se pide crear dos script en Python, así como los comandos necesarios para ejecutarlos (mediante spark-submit) para:

a. Obtener para cada película, la nota media de todas sus votaciones.  
b. Películas cuya nota media sea superior a 3.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Crear el SparkSession existente en Databricks
spark = SparkSession.builder.getOrCreate()

# Leer el archivo ratings.txt y crear un RDD
rdd = sc.textFile("dbfs:/FileStore/Notebook/ratings.txt")

# Convertir el RDD en un DataFrame con las columnas correspondientes
columns = ["pelicula_id", "usuario_id", "calificacion", "timestamp"]
rddPeliculas = rdd.map(lambda linea: linea.split("::"))
dfPeliculas = spark.createDataFrame(rddPeliculas, columns)

# Seleccionar las columnas de interés y convertir calificación a float
dfPeliculas = dfPeliculas.select(
    F.col("pelicula_id"),
    F.col("calificacion").cast("float")
)

# Calcular la nota media por cada película
dfMedia = dfPeliculas.groupBy("pelicula_id").agg(F.avg("calificacion").alias("nota_media"))

# Filtrar las películas cuya nota media es superior a 3
dfMediaSuperior = dfMedia.filter(F.col("nota_media") > 3)

# Mostrar los resultados en Databricks
display(dfMediaSuperior)



pelicula_id,nota_media
296,3.597938144329897
467,3.2028985507246377
675,3.3095238095238093
691,4.174757281553398
829,4.231884057971015
1090,3.3698630136986303
1436,3.9696969696969697
1512,3.235294117647059
1572,3.1777777777777776
125,4.098591549295775


# Proyecto

A continuación planteamos el siguiente proyecto para realizar en clase:

**1.** Tenemos las calificaciones de las asignaturas de [matemáticas](https://tajamar365.sharepoint.com/:t:/s/3405-MasterIA2024-2025/EWlo3ON8o4dLjZAmnJa4E2gBG2zbPGb2_kx0UJaqNiMzCA?e=8XRr7X), [inglés](https://tajamar365.sharepoint.com/:t:/s/3405-MasterIA2024-2025/EWbI5WURy9RMk5kPpMVU6HoBIKSpzjWjdSz982dWS3nboQ?e=C5TONj) y [física](https://tajamar365.sharepoint.com/:t:/s/3405-MasterIA2024-2025/EYNDTyKO2UdOt2t3GPKCZ0UBt9-lasNRDclinzD-dCleBw?e=l9MIiu) de los alumnos del instituto en 3 documentos de texto. A partir de estos ficheros:

a. Crea 3 RDD de pares, uno para cada asignatura, con los alumnos y sus notas  



In [0]:
# Crear RDDs de los archivos con filtrado para evitar errores
rddMates = (
    sc.textFile("dbfs:/FileStore/Notebook/notas_mates.txt")
    .filter(lambda linea: len(linea.split(",")) == 2)  # Filtrar líneas con formato correcto
    .map(lambda linea: linea.split(","))
    .map(lambda x: (x[0], float(x[1].strip())))  # Convertir la nota a float y eliminar espacios extra
)

rddIngles = (
    sc.textFile("dbfs:/FileStore/Notebook/notas_ingles.txt")
    .filter(lambda linea: len(linea.split(",")) == 2)  # Filtrar líneas con formato correcto
    .map(lambda linea: linea.split(","))
    .map(lambda x: (x[0], float(x[1].strip())))  # Convertir la nota a float y eliminar espacios extra
)

rddFisica = (
    sc.textFile("dbfs:/FileStore/Notebook/notas_fisica.txt")
    .filter(lambda linea: len(linea.split(",")) == 2)  # Filtrar líneas con formato correcto
    .map(lambda linea: linea.split(","))
    .map(lambda x: (x[0], float(x[1].strip())))  # Convertir la nota a float y eliminar espacios extra
)





b. Crea un solo RDD con todas las notas  


In [0]:
rddTodasLasNotas = rddMates.union(rddIngles).union(rddFisica)


c. ¿Cuál es la nota más baja que ha tenido cada alumno?  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession
spark = SparkSession.builder.getOrCreate()

# RDD con la nota mínima de cada alumno
notaMinimaPorAlumno = rddTodasLasNotas.reduceByKey(lambda a, b: min(a, b))

# Convertir el RDD en un DataFrame
dfNotaMinima = notaMinimaPorAlumno.map(lambda x: Row(alumno=x[0], nota_minima=x[1])).toDF()

# Mostrar el DataFrame en formato tabla
display(dfNotaMinima)



alumno,nota_minima
Angel,4.0
Carlos,4.0
Anabel,2.0
Jorge,5.0
Susana,2.0
Andres,4.0
Jose Juan,3.0
Rocio,4.0
Fernando,5.0
Oscar,3.0


d. ¿Cuál es la nota media de cada alumno?  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (usualmente ya existe en Databricks, por lo que puedes reutilizarlo)
spark = SparkSession.builder.getOrCreate()

# Convertir el RDD con la nota media en un DataFrame
dfNotaMedia = notaMediaPorAlumno.map(lambda x: Row(alumno=x[0], nota_media=round(x[1], 2))).toDF()

# Mostrar el DataFrame en formato tabla
display(dfNotaMedia)



alumno,nota_media
Angel,6.33
Carlos,5.33
Anabel,5.67
Jorge,6.67
Susana,6.67
Andres,4.67
Jose Juan,3.67
Rocio,5.5
Fernando,7.0
Oscar,5.0


e. ¿Cuantos estudiantes suspende cada asignatura?  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (si no existe ya)
spark = SparkSession.builder.getOrCreate()

# Contar suspensos en cada asignatura
suspensosMates = rddMates.filter(lambda x: x[1] < 5).count()
suspensosIngles = rddIngles.filter(lambda x: x[1] < 5).count()
suspensosFisica = rddFisica.filter(lambda x: x[1] < 5).count()

# Crear un RDD con los resultados para convertir a DataFrame
rddSuspensos = sc.parallelize([
    ("Matemáticas", suspensosMates),
    ("Inglés", suspensosIngles),
    ("Física", suspensosFisica)
])

# Convertir el RDD a un DataFrame
dfSuspensos = rddSuspensos.map(lambda x: Row(asignatura=x[0], numero_suspensos=x[1])).toDF()

# Mostrar el DataFrame en formato tabla
display(dfSuspensos)


asignatura,numero_suspensos
Matemáticas,7
Inglés,7
Física,8


f. ¿En qué asignatura suspende más gente?  


In [0]:
suspensos = {
    "Matemáticas": suspensosMates,
    "Inglés": suspensosIngles,
    "Física": suspensosFisica
}
asignaturaMasSuspensos = max(suspensos, key=suspensos.get)
print(f"Asignatura con más suspensos: {asignaturaMasSuspensos}")


Asignatura con más suspensos: Física


g. Total de notables o sobresalientes por alumno, es decir, cantidad de notas superiores o igual a 7.  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (usualmente ya existe en Databricks)
spark = SparkSession.builder.getOrCreate()

# Convertir el RDD con notables y sobresalientes en un DataFrame
dfNotablesSobresalientes = notablesSobresalientes.map(lambda x: Row(alumno=x[0], num_notables_sobresalientes=x[1])).toDF()

# Mostrar el DataFrame en formato tabla
display(dfNotablesSobresalientes)


alumno,num_notables_sobresalientes
Carlos,1
Angel,1
Anabel,2
Jorge,1
Susana,2
Oscar,1
Isabel,3
Fernando,2
Alejandro,1
Rocio,1


h. ¿Qué alumno no se ha presentado a inglés?  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (usualmente ya existe en Databricks)
spark = SparkSession.builder.getOrCreate()

# Convertir el RDD de los alumnos que no se presentaron a inglés en un DataFrame
dfNoPresentadosIngles = noPresentadosIngles.map(lambda x: Row(alumno=x)).toDF()

# Mostrar el DataFrame en formato tabla
display(dfNoPresentadosIngles)



alumno
Pedro


i. ¿A cuántas asignaturas se ha presentado cada alumno?  


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (usualmente ya existe en Databricks)
spark = SparkSession.builder.getOrCreate()

# Convertir el RDD con asignaturas por alumno en un DataFrame
dfAsignaturasPorAlumno = asignaturasPorAlumno.map(lambda x: Row(alumno=x[0], numero_asignaturas=x[1])).toDF()

# Mostrar el DataFrame en formato tabla
display(dfAsignaturasPorAlumno)


alumno,numero_asignaturas
Angel,3
Carlos,3
Anabel,3
Jorge,3
Susana,3
Andres,3
Jose Juan,3
Rocio,4
Fernando,3
Oscar,3


j. Obtén un RDD con cada alumno con sus notas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Crear el SparkSession (usualmente ya existe en Databricks)
spark = SparkSession.builder.getOrCreate()

# Convertir el RDD con asignaturas por alumno en un DataFrame
dfAsignaturasPorAlumno = asignaturasPorAlumno.map(lambda x: Row(alumno=x[0], numero_asignaturas=x[1])).toDF()

# Convertir el RDD con notas por alumno en un DataFrame
dfNotasPorAlumno = notasPorAlumno.map(lambda x: Row(alumno=x[0], notas=x[1])).toDF()

# Mostrar el DataFrame en formato tabla
display(dfAsignaturasPorAlumno)
display(dfNotasPorAlumno)



alumno,numero_asignaturas
Angel,3
Carlos,3
Anabel,3
Jorge,3
Susana,3
Andres,3
Jose Juan,3
Rocio,4
Fernando,3
Oscar,3


alumno,notas
Angel,"List(6.0, 4.0, 9.0)"
Carlos,"List(4.0, 8.0, 4.0)"
Anabel,"List(8.0, 7.0, 2.0)"
Jorge,"List(10.0, 5.0, 5.0)"
Susana,"List(9.0, 2.0, 9.0)"
Andres,"List(4.0, 6.0, 4.0)"
Jose Juan,"List(5.0, 3.0, 3.0)"
Rocio,"List(6.0, 4.0, 5.0, 7.0)"
Fernando,"List(5.0, 7.0, 9.0)"
Oscar,"List(7.0, 3.0, 5.0)"
